<img src="https://upload.wikimedia.org/wikipedia/fr/c/ca/Logo_Ligue_1_Uber_Eats_2020.svg" style="height:150px">
<hr style="border-width:2px;border-color:#75DFC1">
<h2 style = "text-align:center">Extraction de données </h2> 
<hr style="border-width:2px;border-color:#75DFC1">

### API

- PL : "https://api.football-data.org/v4/competitions/PL/matches"
- LIGA : "https://api.football-data.org/v4/competitions/PD/matches"
- Ligue 1 : "https://api.football-data.org/v4/competitions/FL1/matches"
- Ligue des Champions : "https://api.football-data.org/v4/competitions/CL/matches"
- Serie A : "https://api.football-data.org/v4/competitions/SA/matches"
- Bundesliga : "https://api.football-data.org/v4/competitions/BL1/matches"

### EXTRACTION DATES CHAMPIONATS

il faut prendre en compte le changement d'heure d'été si on veut récupérer les bonnes heures. Il faut également ajouter 1h à l'UTC de Flashcore (on ajoute 50 minutes pour être large par rapport à la programmation Canal qui peut commmencer à 20h55 au lieux de 21h par exemple)
Il faut également prendre en compte l'heure française de Flashscore versus l'heure UTC de NAGRA et de la majorité des tables snowflake. Heure fr (flashscore = + 2H vs UTC en heure d'été et +1H vs UTC en heure d'hiver). 

ATTENTION: l'heure NAGRA est en UTC à partir du 1er avril 2023 seulement.
Avant, l'heure est recalculé sur l'heure France

In [5]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to get soccer games for a specific season
def get_soccer_games(season):
    api_key = "fba82e8db8274bb1bdcbdeffd02ac656"
    headers = {"X-Auth-Token": api_key}
    base_url =  "https://api.football-data.org/v4/competitions/FL1/matches"

    response = requests.get(base_url, headers=headers, params={"season": season})
    data = response.json()

    matches = []
    for match in data['matches']:
        league = match['competition']['name']
        home_team = match['homeTeam']['name']
        away_team = match['awayTeam']['name']
        full_datetime = datetime.strptime(match['utcDate'], "%Y-%m-%dT%H:%M:%SZ")

        # New condition: If full_datetime is before April 1st, do not change it
        if full_datetime < datetime(full_datetime.year, 4, 1):
            updated_datetime = full_datetime + timedelta(hours=1)
        else:
            # Check if the date is before or after the daylight saving time change
            if full_datetime < datetime(2023, 3, 26):
                # Add 1 hour to the datetime
                updated_datetime = full_datetime + timedelta(minutes= -10)
            else:
                # Add 2 hours to the datetime
                updated_datetime = full_datetime + timedelta(hours=-1, minutes=50)

        matches.append({"league": league, "home_team": home_team, "away_team": away_team, "full_datetime": updated_datetime})

    return matches

# Example usage
season = "2024"
soccer_games = get_soccer_games(season)
league = "Ligue 1"

for game in soccer_games:
    resultat_ligue1 = f"{game['league']}: {game['home_team']} vs {game['away_team']}: {game.get('full_datetime', 'N/A')}"
    print(resultat_ligue1)

Ligue 1: Stade Brestois 29 vs Olympique de Marseille: 2024-08-17 23:50:00
Ligue 1: Montpellier HSC vs RC Strasbourg Alsace: 2024-08-17 23:50:00
Ligue 1: Toulouse FC vs FC Nantes: 2024-08-17 23:50:00
Ligue 1: Angers SCO vs Racing Club de Lens: 2024-08-17 23:50:00
Ligue 1: AS Monaco FC vs AS Saint-Étienne: 2024-08-17 23:50:00
Ligue 1: Stade de Reims vs Lille OSC: 2024-08-17 23:50:00
Ligue 1: AJ Auxerre vs OGC Nice: 2024-08-17 23:50:00
Ligue 1: Le Havre AC vs Paris Saint-Germain FC: 2024-08-17 23:50:00
Ligue 1: Stade Rennais FC 1901 vs Olympique Lyonnais: 2024-08-17 23:50:00
Ligue 1: Paris Saint-Germain FC vs Montpellier HSC: 2024-08-24 23:50:00
Ligue 1: RC Strasbourg Alsace vs Stade Rennais FC 1901: 2024-08-24 23:50:00
Ligue 1: FC Nantes vs AJ Auxerre: 2024-08-24 23:50:00
Ligue 1: AS Saint-Étienne vs Le Havre AC: 2024-08-24 23:50:00
Ligue 1: Lille OSC vs Angers SCO: 2024-08-24 23:50:00
Ligue 1: OGC Nice vs Toulouse FC: 2024-08-24 23:50:00
Ligue 1: Racing Club de Lens vs Stade Brestois 29

Récupérer la fin des matchs, ajout de 105 minutes (durée moyenne d'un match de football 115 minutes) à la date de début de diffusion. On ajoute 105 pour eviter les rediffusion qui chevauchent parfois la fin d'un match

In [7]:
##resultat_ligue1.to_csv('FootL1.csv', index=False)

In [8]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to get soccer games for a specific season
def get_soccer_games(season):
    api_key = "fba82e8db8274bb1bdcbdeffd02ac656"
    headers = {"X-Auth-Token": api_key}
    base_url =  "https://api.football-data.org/v4/competitions/PD/matches"

    response = requests.get(base_url, headers=headers, params={"season": season})
    data = response.json()

    matches = []
    for match in data['matches']:
        league = match['competition']['name']
        home_team = match['homeTeam']['name']
        away_team = match['awayTeam']['name']
        full_datetime = datetime.strptime(match['utcDate'], "%Y-%m-%dT%H:%M:%SZ")

        # New condition: If full_datetime is before April 1st, do not change it
        if full_datetime < datetime(full_datetime.year, 4, 1):
            updated_datetime = full_datetime + timedelta(hours=1, minutes=95)
        else:
            # Check if the date is before or after the daylight saving time change
            if full_datetime < datetime(2023, 3, 26):
                # Add 1 hour to the datetime
                updated_datetime = full_datetime + timedelta(minutes=85)
            else:
                # Add 2 hours to the datetime
                updated_datetime = full_datetime + timedelta(hours=-1, minutes=145)

        matches.append({"league": league, "home_team": home_team, "away_team": away_team, "full_datetime": updated_datetime})

    return matches

# Example usage
season = "2022"
soccer_games = get_soccer_games(season)
league = "Primera Division"

for game in soccer_games:
    resultat_ligue1 = f"{game['league']}: {game['home_team']} vs {game['away_team']}: {game.get('full_datetime', 'N/A')}"
    print(resultat_ligue1)

Primera Division: CA Osasuna vs Sevilla FC: 2022-08-12 20:25:00
Primera Division: RC Celta de Vigo vs RCD Espanyol de Barcelona: 2022-08-13 16:25:00
Primera Division: Real Valladolid CF vs Villarreal CF: 2022-08-13 18:25:00
Primera Division: FC Barcelona vs Rayo Vallecano de Madrid: 2022-08-13 20:25:00
Primera Division: Cádiz CF vs Real Sociedad de Fútbol: 2022-08-14 16:55:00
Primera Division: Valencia CF vs Girona FC: 2022-08-14 18:55:00
Primera Division: UD Almería vs Real Madrid CF: 2022-08-14 21:25:00
Primera Division: Athletic Club vs RCD Mallorca: 2022-08-15 16:55:00
Primera Division: Getafe CF vs Club Atlético de Madrid: 2022-08-15 18:55:00
Primera Division: Real Betis Balompié vs Elche CF: 2022-08-15 20:55:00
Primera Division: RCD Espanyol de Barcelona vs Rayo Vallecano de Madrid: 2022-08-19 19:25:00
Primera Division: Sevilla FC vs Real Valladolid CF: 2022-08-19 21:25:00
Primera Division: CA Osasuna vs Cádiz CF: 2022-08-20 16:55:00
Primera Division: RCD Mallorca vs Real Betis B